# Brainstorm

I went over the first iteration of my api design with my advisor and he recommended that I changed a few things about the design. His suggestions were mostly related to the diagram that I had drawn up, and we both decided that it would be good to focus on getting the diagram right and not focus so much on defining the api as it is going to be subject to a lot of changes. It would be too much work to continually update the design, and the implementation is most likely going to change the design substantially. That being said, here are the problems that my advisor pointed out with my previous diagram.  

1. It is not clear what exactly the colored boxes are. Some boxes indicate a code package, whereas others indicate an actual data object. The semantics of the colored boxes need to be clearly defined. 
2. There are multiple paths through the diagram, which correspond with the different scripts that are going to run for the experiments. It would be much easier to understand the diagram (and therefore the experiment code), if there were multiple diagrams with a 1:1 mapping between diagram and script. 

Both of these problems need to be addressed. 

## 1. Define semantics of colored box 
The solution could be as simple as defining a circle as a data object and a box as a code module object. I've tried this and it is looking better. One problem that I am having is a lack of definition for the data sources and what there names are. From initial thinking, there are 4 different types of data 

1. Training data. This includes both training input and training output
2. Validation data. This is exaclty the same as training but at a smaller size. 
3. Test data. This is also the same as training, but at a smaller size. 
4. Production data. This is a different data source in that there is no output to compare against. All it is is a MusicXML file. 

This data can be categorized as development data and production data. 

* Development Data 
    * Train, validation, test split in a json file 
    * Input
        * MusicXML. Comes from raw data 
    * Output
        * MIDI: Comes from both model and raw data. 
        * Used for quantitative evaluation
* Production Data 
    * Input
        * MusicXML. Comes from raw data 
    * Output
        * Comes only from model. 
        * Used for qualitative evaluation  



## Additional thoughts

After taking an even closer look at the data processing code I think that I want to add an additional step in the pipeline that will make thinking about the abstractions easier. The code currently has a large amount of code that is dedicated to pre-processing the data with a score-performance alignment algorithm. This is the data that is used to create the feature representations of the data which are then ultimately fed into the model. All of the code for the score-performance alignment seems to be working well, but (as is the case with all of the code in the project) it is extremely hard to follow and engineered poorly. This will be an additional step that I'll want to add into my framework, and I may end up re-writing some of the code invovled with this as well. Even if I don't, it will be good to understand what is happening from an abstract level so that I can think about it correctly in the development of the experiment. 

# New Design
After brainstorming some more ideas and drawing new diagrams, I think that I have a better design and mental model for what the system is going to look like and how to implement it. I went ahead and split out the single diagram into mulitple diagrams to represent the different jobs that need to run to facilitate the entire system. I'll list each of them out and explain them where needed. The same categorization of data, models, and evaluation still applies. 

## Data
There are two data jobs that I have created. The first deals with intial processing of the data, and the second deals with the getting that intermediate form to a featurized version. 

### Job 1: Run score-performance alignment algorithm and save data in an intermediate state 
The reason that I added this job was because it already exists in the system. The way that the current system works is that a score-performance alignment algorithm is run on the data and then saved in files that give alignment information about every single note. These files are then used with the raw MusicXML and MIDI files and converted to feature form. As can be seen from the graph, the job is as simple as reading in raw data, running a pre-processing job, and then writing that data back out. 

![First diagram](data-1.png)

### Job 2: Featurize the intermediate and raw form 
This jobs is the most necessary for processing data. It will take it both the metdata about the score alignment as well as the raw data and featurize it into a form that can be fed into a model. 

![Second Diagram](data-2.png)

## Models
There is only model job and that consists of running a training job. Outside of training, everything else that has to do with the model is left in the evalaution phase. The model training job is the most complicated. 

![Third Diagram](model-1.png)

## Evaluation 
There are 3 evaluation jobs. One for the quantitative evauluation, and two for the qualitative evaluation. The qualitative evaluation is separated into two jobs mostly becuase one part involves an automated script and the other involves a human interaction and feedback process. 

### Job 1: Quantitative Evaluation 
This job will take the best trained model given from the model training job and feed in the test data. It will then calculate any relevant evaluation metrics and present them for interpretation 

![Fourth diagram](evaluation-1.png)

### Job 2: Qualitative Evaluation: Generating Raw Output Data 
The point of this job is simply to generate raw output data that can then be extracted by a human and given to a system that will allow for human feedback. This job takes in a trained model with production data and generates an actual midi file that represents the performance. The outputted midi file will then be used for the qualitative evaluation as part of a separate process. 

![Fifth diagram](evaluation-2.png)

### Job 3: Qualitative Evaluation: Human feedback 
This job has double outlines around processes that don't represent an automated script, but involve interaction with humans. The process is simple; take the raw output from the model, present it to humans, gather feedback, and then present that feedback to a job that can process the results. 

![Sixth diagram](evaluation-3.png)

